<a href="https://colab.research.google.com/github/Dheeraj13082004/Alzheimer-s-Detection-../blob/main/Alzheimers_Disease_classification_EfficientNetB6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'alzheimersdisease5classdatasetadni:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1161732%2F1946782%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240423%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240423T184634Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2541eef89cf5144148fab5bd19ea7267bfb6d34cb3602b07474052a3c867761e670469629d07558a78c8a8bac77993b9da6d23bb5d59c29a480c8c862efe64a8be062c3a8c7428e8a067b996fb43c40857847bf386fdc472dc4d7c9b717967726745361a416d9c04a446ae2552117130fb113d3ade60861cbff848df9d56313a7978b5fb9dce3bf767d2e0c8034711a70ed40d05dfc4c405904c4679c3c6d15e40a9359e5ee123ef29a92fced87925576b19bf95d65752de8350c2b49c3580ec85fa91d646fe84aef989e80cdfcf21b04d530a225494d7f97097b4c50e3b473b2e9fb852e72bd7fd956ff8c93a46b139e7bb7f6802eb8c68dca186bdc29466de'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 17718755 bytes downloaded
Downloaded and uncompressed: alzheimersdisease5classdatasetadni
Data source import complete.


In [3]:
import tensorflow as tf
import pandas as pd

from tensorflow.keras.callbacks  import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
import os
from tqdm import tqdm
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from PIL import Image

import ipywidgets as widgets
import io
from IPython.display import clear_output,display


# Read Data

In [4]:
image_path="../input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/train/"

In [5]:
cls_name=os.listdir(image_path)
print(cls_name)

['Final AD JPEG', 'Final CN JPEG', 'Final LMCI JPEG', 'Final MCI JPEG', 'Final EMCI JPEG']


In [6]:
print("Number of classes : {}".format(len(cls_name)))

Number of classes : 5


In [7]:
number_of_images={}
for class_name in cls_name:
    number_of_images[class_name]=len(os.listdir(image_path+"/"+class_name))
images_each_class=pd.DataFrame(number_of_images.values(),index=number_of_images.keys(),columns=["Number of images"])
images_each_class

,Number of images
Final AD JPEG,145
Final CN JPEG,493
Final LMCI JPEG,61
Final MCI JPEG,198
Final EMCI JPEG,204


In [ ]:
label_name=["Final AD JPEG","Final CN JPEG","Final EMCI JPEG","Final LMCI JPEG","Final MCI JPEG"]

# train data and test data preprocessing

In [8]:
print("presprocess train data\n")

image_data=[]
label_data=[]
image_size=150

for i in label_name:
    data_path=os.path.join("../input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/","train",i)
    for m in tqdm(os.listdir(data_path)):
        image=cv2.imread(os.path.join(data_path,m))
        image=cv2.resize(image,(image_size,image_size))

        image_data.append(image)
        label_data.append(i)





for i in label_name:
    data_path=os.path.join("../input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/","test",i)
    for m in tqdm(os.listdir(data_path)):
        image=cv2.imread(os.path.join(data_path,m))
        image=cv2.resize(image,(image_size,image_size))

        image_data.append(image)
        label_data.append(i)

image_data=np.array(image_data)
label_data=np.array(label_data)

presprocess train data



NameError: name 'label_name' is not defined

In [ ]:
image_data.shape

In [ ]:
image_data,label_data=shuffle(image_data,label_data,random_state=42)

# Split Data And Label

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(image_data,label_data,test_size=0.2,random_state=42)

# apply onehot encoding

In [ ]:
train_label_data_new=[]
test_label_data_new=[]
for n in Y_train:
    train_label_data_new.append(label_name.index(n))
Y_train=train_label_data_new
Y_train=to_categorical(Y_train)

for n in Y_test:
    test_label_data_new.append(label_name.index(n))
Y_test=test_label_data_new
Y_test=to_categorical(Y_test)





In [ ]:
Y_train

# Create Model

In [ ]:
efficientNetB6=tf.keras.applications.efficientnet.EfficientNetB6(weights="imagenet",include_top=False,input_shape=(150,150,3))
model=efficientNetB6.output
model=tf.keras.layers.GlobalAveragePooling2D()(model)
model=tf.keras.layers.Dropout(0.5)(model)
model=tf.keras.layers.Dense(5,activation="softmax")(model)
model=tf.keras.models.Model(inputs=efficientNetB6.input,outputs=model)
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()


In [ ]:
tensorboard = TensorBoard(log_dir="logs")
checkpoint = ModelCheckpoint("efficient_B6.h5",
                            monitor='val_accuracy',
                            mode = "auto",
                            verbose = 1,
                            save_best_only =True)

In [ ]:
reduce_LR = ReduceLROnPlateau(monitor="val_accuracy",
                             factor = 0.3,
                             patience = 2,
                             min_delta=0.001,
                             mode = "auto",
                            verbose = 1)

In [ ]:
history = model.fit(X_train, Y_train,
                  validation_data =(X_test,Y_test),
                  epochs =80,
                  verbose =1,
                  batch_size = 10,
                  callbacks = [tensorboard, checkpoint, reduce_LR])

In [ ]:
result= model.evaluate(X_test,Y_test,batch_size=10)
print("Accuracy is\n",result)



# history for accuracy

In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

 # history for loss

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# %load_ext tensorboard

In [ ]:
#  %reload_ext tensorboard

In [ ]:
# %tensorboard --logdir logs/fit

# Model Prediction

In [ ]:
prediction=model.predict(X_test)

pred_label=np.argmax(prediction,axis=1)

y_pred=np.argmax(Y_test,axis=1)



# Confusion Matrix

In [ ]:
cm = confusion_matrix(y_pred,pred_label)
sns.heatmap(cm,  annot=True, fmt="d" ,cmap="Oranges",xticklabels=label_name,yticklabels=label_name)
plt.title("Confusion_matrix\n")
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Classification Report

In [ ]:
print(classification_report(y_pred, pred_label,target_names=["Final AD JPEG" , "Final CN JPEG",  "Final EMCI JPEG","Final LMCI JPEG","Final MCI JPEG"]))

# Data Prediction

In [ ]:
def image_prediction(upload):
    for name, fileinfo in uploader.value.items():
        image_file=Image.open(io.BytesIO(fileinfo["content"]))
    predict_image=cv2.cvtColor(np.array(image_file),cv2.COLOR_RGB2BGR)
    predict_image=cv2.resize(predict_image,(150,150))
    predict_image=predict_image.reshape(1,150,150,3)
    predict_image=model.predict(predict_image)
    predict_image=np.argmax(predict_image,axis=1)[0]

    if predict_image==0:
        predict_image="Final AD JPEG"
    elif predict_image==1:
        predict_image="Final CN JPEG"
    elif predict_image==2:
        predict_image="Final EMCI JPEG"
    elif predict_image==3:
        predict_image="Final LMCI JPEG"
    elif predict_image==4:
        predict_image="Final MCI JPEG"

    if predict_image !=1:
        print(f"Model prediction is  {predict_image}")



# Upload Data For Prediction

In [ ]:
uploader=widgets.FileUpload()
display(uploader)

In [ ]:
button=widgets.Button(description="Predict")
out_put=widgets.Output()

def button_click(_):
    with out_put:
        clear_output()
        try:
            image_prediction(uploader)
        except:
            print("Enter Correct Image File")

button.on_click(button_click)
widgets.VBox([button,out_put])